# 概念

对MLP使用平移不变性和局部性的到卷积层

核矩阵和偏移是可学习的参数
核的大小是超参

# 卷积操作

In [1]:
import torch 
from torch import nn

/opt/homebrew/Caskroom/miniforge/base/envs/Atari/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
def corr2d(X, K):
    h, w = K.shape
    y = torch.zeros((X.shape[0] - h + 1), X.shape[1]-w+1)
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return y

In [37]:
X = torch.tensor(range(9)).reshape((3,3))
K = torch.tensor(range(4)).reshape((2,2))

In [38]:
X, K

(tensor([[0, 1, 2],
         [3, 4, 5],
         [6, 7, 8]]),
 tensor([[0, 1],
         [2, 3]]))

In [39]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

# 二维卷积层

In [40]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias # 广播机制
        

In [41]:
X = torch.ones((6,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [42]:
K = torch.tensor([[1.0, -1.0]])
K.shape

torch.Size([1, 2])

In [44]:
Y = corr2d(X, K) # 边缘检测
Y.shape
# 我们拿这个Y作为真值，之后来学一下

torch.Size([6, 7])

# 学习卷基层的权重

In [46]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False)

X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch: {i + 1}, loss: {l.sum():.3f}')

batch: 2, loss: 3.729
batch: 4, loss: 0.965
batch: 6, loss: 0.301
batch: 8, loss: 0.107
batch: 10, loss: 0.041


In [47]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9716, -1.0126]])